In [490]:
import base64
import datetime
import requests
import json
import gpxpy

import gpxpy.gpx
import folium

In [491]:
gpx_file = open('lora.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

In [492]:
lat_gw, lon_gw = -33.11341233, -64.32847861

In [505]:
#url_data = 'http://0.0.0.0:5000/api/mqtt/1'
#data = requests.get(url_data).json()
with open('mqtt_payloads.json') as json_file:
    data = json.load(json_file)

In [494]:
coord = []
deltas = []
for waypoint in gpx.waypoints:
    if waypoint.name == 'Parada de autobús':
        coord.append((waypoint.latitude, waypoint.longitude))
        delta = abs(lat_gw-waypoint.latitude + lon_gw-waypoint.longitude)
        deltas.append(delta)
offset_delta = deltas.index(min(deltas))

In [495]:
rssi_values = []
for entry in data:
        rssi = entry['payload']['metadata']['gateways'][0]['rssi']
        rssi_values.append(rssi)
offset_rssi = rssi_values.index(max(rssi_values))

In [496]:
coord_map = coord[offset_delta::] + coord[0:offset_delta]
rssi_map = rssi_values[offset_rssi::] + rssi_values[0:offset_rssi]

In [497]:
map_tuples = []
for i, rssi in enumerate(rssi_values):
    try:
        lat, lon = coord_map[i]
        map_tuples.append((lat, lon, rssi))
    except:
        pass

In [498]:
min_rssi = min(rssi_values)
max_rssi = max(rssi_values)
min_rssi, max_rssi

(-122, -66)

In [499]:
def color(rssi):
    min_rssi = min(rssi_values)
    max_rssi = max(rssi_values)
    chunks_len = len(rssi_values) // 4
    if rssi in range(min_rssi, rssi_values[chunks_len]): 
        col = 'red'
    elif rssi in range(rssi_values[chunks_len], rssi_values[2*chunks_len]): 
        col = 'orange'
    elif rssi in range(rssi_values[2*chunks_len], max_rssi):
        col = 'lightgreen'
    else: 
        col='green'
    return col 

In [509]:
my_map = folium.Map(location=[map_tuples[0][0], 
                              map_tuples[0][1]], 
                    zoom_start=14)

for tup in map_tuples:
    lat, lon = tup[0], tup[1]
    rssi = tup[2]
    # Marker() takes location coordinates  
    # as a list as an argument 
    folium.Marker(location=[lat,lon], 
                  popup=rssi, 
                  icon=folium.Icon(color=color(rssi))).add_to(my_map)
    folium.Marker(location=[lat_gw, lon_gw],
                 popup='Gateway',
                 icon=folium.Icon(color='black')).add_to(my_map)
my_map.save("./lora_coverage_map.html")